Based on http://machinelearningmastery.com/time-series-forecast-study-python-monthly-sales-french-champagne/

In [1]:
%matplotlib inline

In [2]:
import requests
from pandas.io.json import json_normalize
import pandas as pd

In [3]:
sensorId = 4727
r = requests.get('http://api.gios.gov.pl/pjp-api/rest/data/getData/' + str(sensorId))

In [15]:
concentration = json_normalize(r.json())
concentrationFrame = pd.DataFrame()
concentrationFrame["dates"] = [d[u'date'] for d in concentration["values"].values.item()]
concentrationFrame["values"] = [d[u'value'] for d in concentration["values"].values.item()]
concentrationFrame.set_index(["dates"], inplace=True)
concentrationFrame = concentrationFrame.iloc[::-1]
concentrationFrame = concentrationFrame[:-1]  #removing last line

In [24]:
validationFrame = concentrationFrame.tail(n=len(concentrationFrame)/10)
trainingFrame = concentrationFrame[:-len(validationFrame)]

In [27]:
trainingFrame

,values
dates,
2017-04-20 01:00:00,26.20380
2017-04-20 02:00:00,21.36130
2017-04-20 03:00:00,15.92620
2017-04-20 04:00:00,9.71458
2017-04-20 05:00:00,7.08664
2017-04-20 06:00:00,1.00000
2017-04-20 07:00:00,15.98260
2017-04-20 08:00:00,30.82270
2017-04-20 09:00:00,17.07150


In [28]:
validationFrame

,values
dates,
2017-04-22 03:00:00,12.5559
2017-04-22 04:00:00,16.6114
2017-04-22 05:00:00,18.7686
2017-04-22 06:00:00,18.0841
2017-04-22 07:00:00,13.1477


In [31]:
print('Whole dataset %d, training %d, validation %d' % (len(concentrationFrame), len(trainingFrame), 
                                                        len(validationFrame)))

Whole dataset 55, training 50, validation 5
